In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import torch
import matplotlib.pyplot as plt
import glob
import cv2

In [37]:
os.chdir('C:\\Users\Quang\\OneDrive\Desktop\\hcl-traffic-light-detection')
DATA_PATH = './traffic light data/'
os.listdir(DATA_PATH)

['Annotations',
 'daySequence1',
 'daySequence2',
 'dayTrain',
 'nightSequence1',
 'nightSequence2',
 'nightTrain',
 'sample-dayClip6',
 'sample-nightClip1']

In [38]:
DAY_TRAIN_PATH = 'C:\\Users\\Quang\\OneDrive\\Desktop\\hcl-traffic-light-detection\\traffic light data\\Annotations\\Annotations\\dayTrain'

In [4]:
os.listdir(DAY_TRAIN_PATH)

['dayClip1',
 'dayClip10',
 'dayClip11',
 'dayClip12',
 'dayClip13',
 'dayClip2',
 'dayClip3',
 'dayClip4',
 'dayClip5',
 'dayClip6',
 'dayClip7',
 'dayClip8',
 'dayClip9']

In [39]:
train_day = []

# df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'dayClip1','frameAnnotationsBOX.csv'),sep=';')
df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'dayClip2','frameAnnotationsBOX.csv'),sep=';')
# df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'dayClip3','frameAnnotationsBOX.csv'),sep=';')

In [40]:
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,dayTraining/dayClip2--00000.jpg,stop,902,242,929,287,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0
1,dayTraining/dayClip2--00000.jpg,stop,1220,334,1244,374,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0
2,dayTraining/dayClip2--00000.jpg,stopLeft,754,243,784,288,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0
3,dayTraining/dayClip2--00000.jpg,stopLeft,174,351,192,378,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,0
4,dayTraining/dayClip2--00001.jpg,stop,902,240,929,289,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,1,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,1
...,...,...,...,...,...,...,...,...,...,...
2168,dayTraining/dayClip2--00974.jpg,go,986,211,1040,301,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,974,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,974
2169,dayTraining/dayClip2--00975.jpg,go,1034,199,1094,299,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,975,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,975
2170,dayTraining/dayClip2--00976.jpg,go,1084,203,1144,293,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,976,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,976
2171,dayTraining/dayClip2--00977.jpg,go,1132,183,1198,288,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,977,dayTraining/dayClip2/dayClip2Shutter0.000800-G...,977


In [41]:
# Duplicate Columns
np.all(df['Origin file'] == df['Origin track']), np.all(df['Origin frame number'] == df['Origin track frame number'])

(True, True)

In [42]:
# Dropping duplicate columns & "Origin file" as we don't need it
df = df.drop(['Origin file','Origin track','Origin track frame number'],axis=1)
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number
0,dayTraining/dayClip2--00000.jpg,stop,902,242,929,287,0
1,dayTraining/dayClip2--00000.jpg,stop,1220,334,1244,374,0
2,dayTraining/dayClip2--00000.jpg,stopLeft,754,243,784,288,0
3,dayTraining/dayClip2--00000.jpg,stopLeft,174,351,192,378,0
4,dayTraining/dayClip2--00001.jpg,stop,902,240,929,289,1
...,...,...,...,...,...,...,...
2168,dayTraining/dayClip2--00974.jpg,go,986,211,1040,301,974
2169,dayTraining/dayClip2--00975.jpg,go,1034,199,1094,299,975
2170,dayTraining/dayClip2--00976.jpg,go,1084,203,1144,293,976
2171,dayTraining/dayClip2--00977.jpg,go,1132,183,1198,288,977


In [43]:
print("Number of Unique Images: ",df.Filename.nunique(),'/',df.shape[0])

Number of Unique Images:  954 / 2173


In [44]:
def changeFilename(x):
    filename = x.Filename
    splitted = filename.split('/')
    clipName = splitted[-1].split('--')[-1]
    return os.path.join(clipName)

df['Filename'] = df.apply(changeFilename, axis=1)

In [45]:
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number
0,00000.jpg,stop,902,242,929,287,0
1,00000.jpg,stop,1220,334,1244,374,0
2,00000.jpg,stopLeft,754,243,784,288,0
3,00000.jpg,stopLeft,174,351,192,378,0
4,00001.jpg,stop,902,240,929,289,1
...,...,...,...,...,...,...,...
2168,00974.jpg,go,986,211,1040,301,974
2169,00975.jpg,go,1034,199,1094,299,975
2170,00976.jpg,go,1084,203,1144,293,976
2171,00977.jpg,go,1132,183,1198,288,977


In [46]:
# We will change annotations to only -> stop (RED), go (GREEN) & warning (YELLOW)
label_to_idx = {'go':1, 'warning':2, 'stop': 3}
idx_to_label = {v:k for k,v in label_to_idx.items()}

def changeAnnotation(x):
    if 'go' in x['Annotation tag']:
        return label_to_idx['go']
    elif 'warning' in x['Annotation tag']:
        return label_to_idx['warning']
    elif 'stop' in x['Annotation tag']:
        return label_to_idx['stop']
    
df['Annotation tag'] = df.apply(changeAnnotation,axis=1)

annotation_tags = df['Annotation tag'].unique()
annotation_tags

array([3, 1, 2], dtype=int64)

In [47]:
df.tail()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number
2168,00974.jpg,1,986,211,1040,301,974
2169,00975.jpg,1,1034,199,1094,299,975
2170,00976.jpg,1,1084,203,1144,293,976
2171,00977.jpg,1,1132,183,1198,288,977
2172,00978.jpg,1,1183,188,1246,278,978


In [48]:
# Changing Column Names
df.columns = ['image_id','label','x_min','y_min','x_max','y_max','frame']
df

,image_id,label,x_min,y_min,x_max,y_max,frame
0,00000.jpg,3,902,242,929,287,0
1,00000.jpg,3,1220,334,1244,374,0
2,00000.jpg,3,754,243,784,288,0
3,00000.jpg,3,174,351,192,378,0
4,00001.jpg,3,902,240,929,289,1
...,...,...,...,...,...,...,...
2168,00974.jpg,1,986,211,1040,301,974
2169,00975.jpg,1,1034,199,1094,299,975
2170,00976.jpg,1,1084,203,1144,293,976
2171,00977.jpg,1,1132,183,1198,288,977


In [49]:
print("Number of Unique Images: ",df.image_id.nunique(),'/',df.shape[0])

Number of Unique Images:  954 / 2173


In [50]:
df

,image_id,label,x_min,y_min,x_max,y_max,frame
0,00000.jpg,3,902,242,929,287,0
1,00000.jpg,3,1220,334,1244,374,0
2,00000.jpg,3,754,243,784,288,0
3,00000.jpg,3,174,351,192,378,0
4,00001.jpg,3,902,240,929,289,1
...,...,...,...,...,...,...,...
2168,00974.jpg,1,986,211,1040,301,974
2169,00975.jpg,1,1034,199,1094,299,975
2170,00976.jpg,1,1084,203,1144,293,976
2171,00977.jpg,1,1132,183,1198,288,977


## Rename all images in the dataset

In [6]:
os.getcwd()

'C:\\Users\\Quang\\OneDrive\\Desktop\\hcl-traffic-light-detection\\yolov5'

In [54]:
# collection = 'C:\\Users\\Quang\\OneDrive\\Desktop\\hcl-traffic-light-detection\\datasets\\train\\images\\'
collection = 'C:\\Users\\Quang\\OneDrive\\Desktop\\hcl-traffic-light-detection\\datasets\\val\\images\\'
# collection = '/Users/quangnguyen/Desktop/hcl/project/data/test/images/'
print(len(os.listdir(collection)))

1031


In [55]:
import os

for i, filename in enumerate(sorted(os.listdir(collection))):
    new_filename = filename.split('--')[-1]
    os.rename(collection + filename, collection + new_filename)

## reformat labeled data

In [56]:
# convert to yolo format
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [57]:
## Write dataframe to textfile

# os.chdir('C:\\Users\\Quang\\OneDrive\\Desktop\\hcl-traffic-light-detection\\datasets\\train\\labels')
os.chdir('C:\\Users\\Quang\\OneDrive\\Desktop\\hcl-traffic-light-detection\\datasets\\val\\labels')
# os.chdir('/Users/quangnguyen/Desktop/hcl/project/data/test/labels')
for index, row in df.iterrows():
    filename = row['image_id'].split('.')[0] + '.txt'
    width = 1280
    height = 960
    xmin = float(row['x_min'])
    ymin = float(row['y_min'])
    xmax = float(row['x_max'])
    ymax = float(row['y_max'])
    
        
    b = (xmin, xmax, ymin, ymax)
    x_center, y_center, w, h = convert((width,height), b)


    with open(filename, 'a') as outfile:
        outfile.write(f'0 {x_center} {y_center} {w} {h}\n')
    

## Training

## Configure yaml file

In [1]:
os.chdir('/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/')

In [2]:
%cd yolov5/

/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/yolov5


In [7]:
!python train.py --img 680 --batch 16 --epochs 100 --data /mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/datasets/datasets.yaml --weights /mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/yolov5s.pt --workers 2


train: weights=/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/yolov5s.pt, cfg=, data=/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/datasets/datasets.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=680, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-132-g014acde torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1

### Use the model to predict

In [20]:
os.chdir('/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/prediction images')
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/yolov5/runs/train/exp9/weights/best.pt', force_reload=True)

# save predicted images to folder
for filename in sorted(glob.glob('/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/traffic light data/dayTrain/dayTrain/dayClip2/frames/*.jpg')):
    image = cv2.imread(filename)
    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    results = model(img)
    plt.imsave(filename.split('/')[-1], np.squeeze(results.render()))

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/quang/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-4-25 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


### Create video from images

In [26]:
import cv2
import numpy as np
import glob
%matplotlib inline
os.chdir('/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/')
img_array = []
for filename in sorted(glob.glob('/mnt/c/users/Quang/OneDrive/desktop/hcl-traffic-light-detection/prediction images/*.jpg')):
    img = cv2.imread(filename)
    # img = cv2.cvtColor(img,cv2.cv2.COLOR_RGB2BGR)

    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

    
 
out = cv2.VideoWriter('project3.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
